In [211]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift
from sklearn import preprocessing

In [276]:
data=pd.read_excel('titanic.xls')
original_data=pd.DataFrame.copy(data)
data

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [277]:
data.drop(['body','name'],1,inplace=True)
data.apply(pd.to_numeric, errors='ignore')
data.fillna(0,inplace=True)
data

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,female,29.0000,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,0.9167,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,female,14.5000,1,0,2665,14.4542,0,C,0,0
1305,3,0,female,0.0000,1,0,2665,14.4542,0,C,0,0
1306,3,0,male,26.5000,0,0,2656,7.2250,0,C,0,0
1307,3,0,male,27.0000,0,0,2670,7.2250,0,C,0,0


In [278]:
def handle_non_numerical_data(data):
    columns=data.columns.values
    for column in columns:
        text_digit_vals={}
        def convert_to_int(val):
            return text_digit_vals[val]
        if data[column].dtype!=np.int64 and data[column].dtype!=np.float64:
            column_contents=data[column].values.tolist()
            unique_elements=set(column_contents)#find uniques
            x=0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique]=x
                    x+=1
            data[column]=list(map(convert_to_int,data[column]))
    return data
            

In [279]:
handle_non_numerical_data(data)

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,1,29.0000,0,0,754,211.3375,156,3,1,273
1,1,1,0,0.9167,1,2,539,151.5500,148,3,26,185
2,1,0,1,2.0000,1,2,539,151.5500,148,3,0,185
3,1,0,0,30.0000,1,2,539,151.5500,148,3,0,185
4,1,0,1,25.0000,1,2,539,151.5500,148,3,0,185
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,1,14.5000,1,0,568,14.4542,0,1,0,0
1305,3,0,1,0.0000,1,0,568,14.4542,0,1,0,0
1306,3,0,0,26.5000,0,0,734,7.2250,0,1,0,0
1307,3,0,0,27.0000,0,0,34,7.2250,0,1,0,0


In [280]:
data.drop(['ticket','home.dest'],1,inplace=True)
x=np.array(data.drop(['survived'],1).astype(float))
x=preprocessing.scale(x)
y=np.array(data['survived'])
classifier=MeanShift()
classifier.fit(x)

MeanShift()

In [281]:
labels=classifier.labels_
cluster_centers=classifier.cluster_centers_
original_data['cluster_group']=np.nan
for i, label in enumerate (labels):
    original_data['cluster_group'].iloc[i]=label

D:\Anaconda\envs\python3366\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [282]:
n_clusters_=len(np.unique(labels))
survival_rates={}
for i in range(n_clusters_):
    temp_data=original_data[(original_data['cluster_group']==float(i))]
    survival_cluster=temp_data[(temp_data['survived']==1)]
    survival_rate=len(survival_cluster)/len(temp_data)
    survival_rates[i]=survival_rate
print(survival_rates)

{0: 0.3659539473684211, 1: 0.23809523809523808, 2: 1.0, 3: 0.7636363636363637, 4: 0.0, 5: 1.0, 6: 0.0}


In [305]:
print("First cluster",original_data[(original_data['cluster_group']==0)])

First cluster       pclass  survived                                           name     sex  \
5          1         1                            Anderson, Mr. Harry    male   
6          1         1              Andrews, Miss. Kornelia Theodosia  female   
7          1         0                         Andrews, Mr. Thomas Jr    male   
8          1         1  Appleton, Mrs. Edward Dale (Charlotte Lamson)  female   
9          1         0                        Artagaveytia, Mr. Ramon    male   
...      ...       ...                                            ...     ...   
1304       3         0                           Zabour, Miss. Hileni  female   
1305       3         0                          Zabour, Miss. Thamine  female   
1306       3         0                      Zakarian, Mr. Mapriededer    male   
1307       3         0                            Zakarian, Mr. Ortin    male   
1308       3         0                             Zimmerman, Mr. Leo    male   

       age  s

In [306]:
print("Second cluster",original_data[(original_data['cluster_group']==1)])

Second cluster       pclass  survived                                               name  \
342        2         1   Becker, Mrs. Allen Oliver (Nellie E Baumgardner)   
449        2         1              Hocking, Mrs. Elizabeth (Eliza Needs)   
550        2         1              Richards, Mrs. Sidney (Emily Hocking)   
626        3         0             Andersson, Miss. Ida Augusta Margareta   
629        3         0                        Andersson, Mr. Anders Johan   
632        3         0  Andersson, Mrs. Anders Johan (Alfrida Konstant...   
644        3         0         Asplund, Mr. Carl Oscar Vilhelm Gustafsson   
646        3         1  Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...   
660        3         1              Baclini, Mrs. Solomon (Latifa Qurban)   
810        3         0                             Ford, Mr. William Neal   
811        3         0            Ford, Mrs. Edward (Margaret Ann Watson)   
828        3         0                        Goodwin, Miss. 

In [310]:
print("Third cluster",original_data[(original_data['cluster_group']==2)])

Third cluster      pclass  survived                                               name  \
49        1         1                 Cardeza, Mr. Thomas Drake Martinez   
50        1         1  Cardeza, Mrs. James Warburton Martinez (Charlo...   
183       1         1                             Lesurer, Mr. Gustave J   
302       1         1                                   Ward, Miss. Anna   

        sex   age  sibsp  parch    ticket      fare        cabin embarked  \
49     male  36.0      0      1  PC 17755  512.3292  B51 B53 B55        C   
50   female  58.0      0      1  PC 17755  512.3292  B51 B53 B55        C   
183    male  35.0      0      0  PC 17755  512.3292         B101        C   
302  female  35.0      0      0  PC 17755  512.3292          NaN        C   

    boat  body                                       home.dest  cluster_group  
49     3   NaN  Austria-Hungary / Germantown, Philadelphia, PA            2.0  
50     3   NaN                    Germantown, Philadelphia,

In [307]:
print("Forth cluster",original_data[(original_data['cluster_group']==3)])

Forth cluster       pclass  survived                                               name  \
0          1         1                      Allen, Miss. Elisabeth Walton   
1          1         1                     Allison, Master. Hudson Trevor   
2          1         0                       Allison, Miss. Helen Loraine   
4          1         0    Allison, Mrs. Hudson J C (Bessie Waldo Daniels)   
11         1         1  Astor, Mrs. John Jacob (Madeleine Talmadge Force)   
17         1         1    Baxter, Mrs. James (Helene DeLaudeniere Chaput)   
21         1         1   Beckwith, Mrs. Richard Leonard (Sallie Monypeny)   
23         1         1                              Bidois, Miss. Rosalie   
24         1         1                                  Bird, Miss. Ellen   
28         1         1                             Bissette, Miss. Amelia   
32         1         1                            Bonnell, Miss. Caroline   
35         1         1                           Bowen, Miss. 

In [308]:
print("Fifth cluster",original_data[(original_data['cluster_group']==4)])

Fifth cluster       pclass  survived                            name     sex  age  sibsp  \
1179       3         0           Sage, Mr. John George    male  NaN      1   
1180       3         0  Sage, Mrs. John (Annie Bullen)  female  NaN      1   

      parch    ticket   fare cabin embarked boat  body home.dest  \
1179      9  CA. 2343  69.55   NaN        S  NaN   NaN       NaN   
1180      9  CA. 2343  69.55   NaN        S  NaN   NaN       NaN   

      cluster_group  
1179            4.0  
1180            4.0  


In [309]:
print("Sixth cluster",original_data[(original_data['cluster_group']==5)])

Sixth cluster      pclass  survived                                               name  \
78        1         1  Compton, Mrs. Alexander Taylor (Mary Eliza Ing...   
116       1         1                Fortune, Mrs. Mark (Mary McDougald)   
139       1         1      Graham, Mrs. William Thompson (Edith Junkins)   
253       1         1    Ryerson, Mrs. Arthur Larned (Emily Maria Borie)   

        sex   age  sibsp  parch    ticket      fare            cabin embarked  \
78   female  64.0      0      2  PC 17756   83.1583              E45        C   
116  female  60.0      1      4     19950  263.0000      C23 C25 C27        S   
139  female  58.0      0      1  PC 17582  153.4625             C125        S   
253  female  48.0      1      3  PC 17608  262.3750  B57 B59 B63 B66        C   

    boat  body                        home.dest  cluster_group  
78    14   NaN                     Lakewood, NJ            5.0  
116   10   NaN                     Winnipeg, MB            5.0  
139 

In [311]:
print("Seventh cluster",original_data[(original_data['cluster_group']==6)])

Seventh cluster      pclass  survived                                  name   sex   age  \
3         1         0  Allison, Mr. Hudson Joshua Creighton  male  30.0   
115       1         0                     Fortune, Mr. Mark  male  64.0   
215       1         0            Newell, Mr. Arthur Webster  male  58.0   
252       1         0            Ryerson, Mr. Arthur Larned  male  61.0   
285       1         0                    Straus, Mr. Isidor  male  67.0   
312       1         0            Widener, Mr. George Dunton  male  50.0   
313       1         0             Widener, Mr. Harry Elkins  male  27.0   

     sibsp  parch    ticket      fare            cabin embarked boat   body  \
3        1      2    113781  151.5500          C22 C26        S  NaN  135.0   
115      1      4     19950  263.0000      C23 C25 C27        S  NaN    NaN   
215      0      2     35273  113.2750              D48        C  NaN  122.0   
252      1      3  PC 17608  262.3750  B57 B59 B63 B66        C  Na